In [189]:
%%capture
pip install -r requirements.txt

In [190]:
import re
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA

<h3>ETAPA 0 - Leitura do CSV (com checagens)</h3>

In [191]:
%%capture
CSV_PATH = Path("input.csv")

assert CSV_PATH.exists(), f"Arquivo não encontrado: {CSV_PATH.resolve()}"

# Tentativas de leitura com encodings comuns
encodings_to_try = ["utf-8", "utf-8-sig", "latin-1"]
last_err = None
df = None
for enc in encodings_to_try:
    try:
        df = pd.read_csv(CSV_PATH, encoding=enc)
        print(f"[OK] Lido com encoding: {enc}")
        break
    except Exception as e:
        last_err = e

if df is None:
    raise RuntimeError(f"Falha ao ler CSV. Último erro: {last_err}")

# Mostra um resumo rápido
print("\n[INFO] Formato do DataFrame:", df.shape)
print("[INFO] Primeiras colunas:", list(df.columns[:10]))
print("\n[INFO] Amostra (5 linhas):")
print(df.head(5))

# Helper para localizar colunas mesmo com sufixos (ex.: 'Type (Remove)')
def find_col(cols, key):
    key = key.lower()
    for c in cols:
        if key in c.lower():
            return c
    return None

# Detecta as colunas principais (só para conferência visual nesta etapa)
col_name = find_col(df.columns, "Name")
col_type  = find_col(df.columns, "Type")
col_env   = find_col(df.columns, "Environment")
col_cr    = find_col(df.columns, "CR")
col_size  = find_col(df.columns, "Size")

print("\n[DETECÇÃO DE COLUNAS]")
print("Type:", col_type)
print("Environment:", col_env)
print("CR:", col_cr)
print("Size:", col_size)

<h3>ETAPA 1 - Seleção de colunas essenciais</h3>

In [192]:
%%capture
# Dicionário com nomes detectados (da etapa anterior)
ESSENTIALS = {
    "name": col_name,
    "type": col_type,
    "env": col_env,
    "cr": col_cr,
    "size": col_size
}

print("\n[INFO] Colunas essenciais selecionadas:")
for k, v in ESSENTIALS.items():
    print(f"  {k:>8} -> {v}")

# Cria uma cópia só com essas colunas
df_work = df[[v for v in ESSENTIALS.values() if v is not None]].copy()

print("\n[INFO] DataFrame de trabalho criado.")
print("[INFO] Formato:", df_work.shape)
print("[INFO] Colunas:", list(df_work.columns))
print("\nPrévia:")
print(df_work.head(10))


<h3>ETAPA 2 - Limpeza e transformação dos dados</h3>

In [193]:
%%capture
print("\nETAPA 2 - Iniciando Limpeza e Transformação...")

# Funções Auxiliares
def parse_cr(x):
    # converter valores de CR para float
    s = str(x).strip()
    if s in ("nan", "", "—", "-", "None"):
        return np.nan
    if "/" in s:
        try:
            a, b = s.split("/", 1)
            return float(a) / float(b)
        except:
            pass
    try:
        return float(s)
    except:
        return np.nan

def clean_text_basic(s):
    # Remove parenteses e normaliza o caps
    s = re.sub(r"\(.*?\)", "", str(s))
    s = re.sub(r"\s+", " ", s)
    return s.strip().title()

def normalize_env(env_str):
    # padronizar ambientes (environment)
    parts = [p.strip().title() for p in str(env_str).split(",") if p.strip()]
    if not parts:
        return "Unknown"
    seen, out = set(), []
    for p in parts:
        if p not in seen:
            seen.add(p)
            out.append(p)
    return ", ".join(out)

#Aplicando limpeza
df_work["Type"] = df_work["Type"].apply(clean_text_basic)
df_work["Size"] =  df_work["Size"].apply(clean_text_basic)
df_work["Environment"] = df_work["Environment"].apply(normalize_env)
df_work["CR_float"] = df_work["CR"].apply(parse_cr)

#Tratando valores faltantes
df_work["CR_float"] = df_work["CR_float"].fillna(0.0)
df_work["Environment"] = df_work["Environment"].replace("", "Unknown")
df_work["Type"] = df_work["Type"].replace("", "Unknown")


# Eliminar duplicatas
before = len(df_work)
df_work.drop_duplicates(inplace=True)
after = len(df_work)

print(f"[INFO] Duplicatas removidas: {before - after}")
print("[INFO] Visualização dos dados limpos:")
print(df_work.head(10))

<h3>ETAPA 3 - Transformação (one-hot e multi-one-hot)</h3>

In [194]:
%%capture
print("\nETAPA 3 - Iniciando Transformação (one-hot e multi-one-hot)...")

#ID estavel apos a limpeza
df_work= df_work.reset_index(drop=True)
df_work.insert(0, "MonsterID", df_work.index + 1)

#One-hot de 'Type' e 'Size'
type_dummies = pd.get_dummies(df_work["Type"], prefix="Type")
size_dummies = pd.get_dummies(df_work["Size"], prefix="Size")

#Multi-one-hot de 'Environment'
known_envs = ["Arctic", "Cave" , "Desert", "Dungeon", "Forest", "Hell", "Mountain", "Plains", "Sky", "Underground", "Urban", "Water", "Unknown"]

def env_one_hot(env_str):
    envs = [e.strip() for e in env_str.split(",") if e.strip()]
    return {f"env_{e}": int(e in envs) for e in known_envs}

env_dummies = df_work["Environment"].apply(env_one_hot).apply(pd.Series)

features = pd.concat([
    df_work[["MonsterID", "CR_float"]],
    type_dummies,
    size_dummies,
    env_dummies
], axis=1)

print("[INFO] Preview das features:",features.shape)
print(features.head(5))

<h3>Exportação final</h3>

In [195]:
print("\nETAPA 4 - Exportação final...")

OUT_DIR = Path("outputs")
OUT_DIR.mkdir(exist_ok=True)

monsters_clean_path = OUT_DIR / "monsters_clean.csv"
monsters_train_path = OUT_DIR / "monsters_train.csv"

if "Name (Remove)" in df_work.columns:
    df_work = df_work.rename(columns={"Name (Remove)": "Name"})

if "Name (Remove)" in df_work.columns:
    df_work = df_work.rename(columns={"Name (Remove)": "Name"})

df_clean = df_work[["MonsterID", "Name", "Type", "Environment", "Size", "CR", "CR_float"]]
df_clean.to_csv("outputs/monsters_clean.csv", index=False)

features.to_csv(monsters_train_path, index=False)

print(f"[INFO] Arquivos exportados:")
print(f"  - Limpo -> {monsters_clean_path.resolve()}")
print(f"  - Treino -> {monsters_train_path.resolve()}")


ETAPA 4 - Exportação final...
[INFO] Arquivos exportados:
  - Limpo -> /home/FACULDADE/IA/TP/Kraken/outputs/monsters_clean.csv
  - Treino -> /home/FACULDADE/IA/TP/Kraken/outputs/monsters_train.csv


<h3>Construção da IA</h3>
Usaremos K-Means (Clusterização)

In [196]:
# Carregar o dataset de treino
df_train = pd.read_csv("outputs/monsters_train.csv")

# identificar os argumentos
monster_ids = df_train["MonsterID"]
cols_temat = [c for c in df_train.columns if any(c.lower().startswith(x) for x in ["type_", "env_", "size_"])]
X = df_train[cols_temat].copy()

print(f"[INFO] Número de features usadas para clustering temático: {len(X.columns)}")

# remover colunas raras (monstros únicos em ambientes exóticos)
rare_cols = [c for c in X.columns if X[c].sum() < 3]
if rare_cols:
    print(f"[INFO] Removendo {len(rare_cols)} colunas raras (menos de 3 ocorrências): {rare_cols[:10]}")
    X = X.drop(columns=rare_cols)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# treinar o K-means
kmeans_thematic = KMeans(n_clusters=4, random_state=42, n_init="auto")
labels_thematic = kmeans_thematic.fit_predict(X_scaled)

df_clusters_thematic = pd.DataFrame({
    "MonsterID": monster_ids,
    "cluster": labels_thematic
})

# Ligar dados para leitura
df_clean = pd.read_csv("outputs/monsters_clean.csv")

df_final_refined = df_clean.merge(df_clusters_thematic, on="MonsterID", how="left")

# Amostra dos clusters
for c in sorted(df_final_refined["cluster"].unique()):
    print(f"\n=== Cluster {c} (amostra) ===")
    display(
        df_final_refined[df_final_refined["cluster"] == c]
        .sample(min(5, len(df_final_refined[df_final_refined["cluster"] == c])))
        [["Type", "Environment", "Size", "CR", "CR_float"]]
    )

print("\n[INFO] Novo clustering concluído — sem CR_float nas features.")


[INFO] Número de features usadas para clustering temático: 39
[INFO] Removendo 7 colunas raras (menos de 3 ocorrências): ['Type_Fiend Fiend', 'Type_Humanoid Humanoid', 'Type_Temp', 'Size_Varies', 'env_Arctic', 'env_Desert', 'env_Unknown']

=== Cluster 0 (amostra) ===


,Type,Environment,Size,CR,CR_float
66,Aberration,"Dungeon, Underground",Small,1.00,1.0
751,Undead,"Dungeon, Sky",Medium,3.00,3.0
692,Ooze,"Cave, Dungeon, Water",Medium,3.00,3.0
760,Undead,"Dungeon, Sky",Medium,5.00,5.0
543,Construct,"Dungeon, Urban",Medium,0.50,0.5



=== Cluster 1 (amostra) ===


,Type,Environment,Size,CR,CR_float
339,Humanoid,"Forest, Plains, Urban",Small,0.50,0.5
605,Humanoid,"Forest, Plains, Urban",Medium,4.00,4.0
335,Humanoid,"Forest, Plains, Urban",Medium,0.50,0.5
41,Humanoid,"Forest, Plains, Urban",Medium,3.00,3.0
598,Humanoid,"Forest, Plains, Urban",Medium,1.00,1.0



=== Cluster 2 (amostra) ===


,Type,Environment,Size,CR,CR_float
119,Fiend,Hell,Small,0.13,0.13
149,Fiend,Hell,Medium,3.00,3.00
155,Fiend,Hell,Large,14.00,14.00
717,Fiend,"Hell, Sky",Medium,4.00,4.00
150,Fiend,"Hell, Sky",Large,9.00,9.00



=== Cluster 3 (amostra) ===


,Type,Environment,Size,CR,CR_float
508,Beast,"Forest, Plains",Medium,0.13,0.13
390,Beast,"Forest, Plains",Large,0.50,0.50
799,Monstrosity,"Forest, Mountain",Small,0.50,0.50
200,Dragon,"Mountain, Sky, Underground, Water",Large,6.00,6.00
703,Fey,"Forest, Plains, Sky",Tiny,0.25,0.25



[INFO] Novo clustering concluído — sem CR_float nas features.


<h3>Heurística para calcular valor do grupo e do encontro</h3>

In [197]:
#Calcula o valor do grupo baseado em uma heuristica simples
def calcular_valor_grupo(niveis_jogadores):
    qtd_jogadores = len(niveis_jogadores)
    soma_niveis = sum(niveis_jogadores)
    nivel_medio = soma_niveis / qtd_jogadores

    if nivel_medio >= 5:
        valor_grupo = soma_niveis / 2
    else:
        valor_grupo = soma_niveis / 4

    return {
        "qtd_jogadores": qtd_jogadores,
        "soma_niveis": soma_niveis,
        "nivel_medio": nivel_medio,
        "valor_grupo": valor_grupo
    }

#Avalia a dificuldade do encontro baseado na soma dos ND dos monstros
def avaliar_encontro(monstros_df, valor_grupo):
    if "CR_float" in monstros_df.columns:
        soma_nd = monstros_df["CR_float"].sum()
    else:
        soma_nd = 0.0

    if soma_nd >= valor_grupo:
        classe = "mortal"
    elif soma_nd >= 0.7 * valor_grupo:
        classe = "difícil"
    elif soma_nd >= 0.4 * valor_grupo:
        classe = "médio"
    else:
        classe = "fácil"

    return {
        "soma_nd_monstros": soma_nd,
        "valor_grupo": valor_grupo,
        "avaliacao_encontro": classe
    }

# Seleciona monstros com base no CR alvo
def escolher_por_cr_alvo(df_filtrado, cr_alvo, quantidade_oponentes):
    if "CR_float" not in df_filtrado.columns or df_filtrado["CR_float"].isna().all():
        # fallback: sorteia puro, com repetição se necessário
        if len(df_filtrado) >= quantidade_oponentes:
            return df_filtrado.sample(quantidade_oponentes, replace=False, random_state=None)
        else:
            falta = quantidade_oponentes - len(df_filtrado)
            base = df_filtrado.copy()
            extra = df_filtrado.sample(falta, replace=True, random_state=None)
            return pd.concat([base, extra], ignore_index=True)

    cand = df_filtrado.copy()
    cand["desvio_do_alvo"] = (cand["CR_float"] - cr_alvo).abs()

    # Ordena por distância em relação ao alvo
    cand = cand.sort_values("desvio_do_alvo")

    # Pegamos o TOP_K mais próximos do alvo
    TOP_K = max(quantidade_oponentes * 3, quantidade_oponentes + 2)
    melhores = cand.head(TOP_K)

    # Sorteia os melhores encontrados entre os oponentes
    if len(melhores) >= quantidade_oponentes:
        escolhidos = melhores.sample(quantidade_oponentes, replace=False, random_state=None)
    else:
        falta = quantidade_oponentes - len(melhores)
        base = melhores.copy()
        extra = melhores.sample(falta, replace=True, random_state=None)
        escolhidos = pd.concat([base, extra], ignore_index=True)

    return escolhidos



<h3>Geração dos encontros com base nas escolhas dos atributos</h3>


In [198]:
# Gera um encontro temático e balanceado
def gerar_encontro(
    df_final_refined,
    niveis_jogadores,
    dificuldade,
    tipo="any",
    ambiente="any",
    tamanho="any",
    quantidade_oponentes=3
):

    logs = []

    #dados da party
    info_party = calcular_valor_grupo(niveis_jogadores)
    valor_grupo = info_party["valor_grupo"]

    # fatores de dificuldade escolhidos como multiplicadores
    fator_dif = {
        "facil":      0.3,
        "fácil":      0.3,
        "medio":      0.6,
        "médio":      0.6,
        "dificil":    0.8,
        "difícil":    0.8,
        "impossivel": 1.0,
        "impossível": 1.0,
    }

    dif_key = dificuldade.lower()
    if dif_key not in fator_dif:
        raise ValueError("Dificuldade inválida. Use: facil / medio / dificil / impossivel")

    mult = fator_dif[dif_key]

    #CR base por oponente
    cr_base = valor_grupo / max(1, quantidade_oponentes)

    # CR ajustado pela dificuldade solicitada
    cr_alvo_por_inimigo = cr_base * mult
    logs.append(f"[DEBUG] CR base {cr_base:.2f}, multiplicador '{dificuldade}'={mult} => alvo {cr_alvo_por_inimigo:.2f}")

    #filtros tematicos
    def aplica_filtros(require_tipo, require_amb, require_tam):
        filtro = pd.Series([True] * len(df_final_refined), index=df_final_refined.index)

        if require_tipo and tipo.lower() != "any":
            filtro &= df_final_refined["Type"].str.contains(tipo, case=False, na=False)

        if require_amb and ambiente.lower() != "any":
            filtro &= df_final_refined["Environment"].str.contains(ambiente, case=False, na=False)

        if require_tam and tamanho.lower() != "any":
            filtro &= df_final_refined["Size"].str.contains(tamanho, case=False, na=False)

        return df_final_refined[filtro]
    
    # busca candidatos baseados nos filtros
    candidatos = aplica_filtros(True, True, True)
    if len(candidatos) == 0:
        logs.append("[INFO] Ninguém bateu tipo+ambiente+tamanho. Relaxando tamanho...")
        candidatos = aplica_filtros(True, True, False)

    if len(candidatos) == 0:
        logs.append("[INFO] Ainda nada. Relaxando ambiente também...")
        candidatos = aplica_filtros(True, False, False)

    if len(candidatos) == 0:
        logs.append("[INFO] Ainda nada. Relaxando tipo também...")
        candidatos = aplica_filtros(False, False, False)

    if len(candidatos) == 0:
        logs.append("[AVISO] Nem com relaxamento encontrei nada. Vou usar TODO o dataset.")
        candidatos = df_final_refined.copy()

    #randomiza o cluster temático para criar variedade
    if "cluster" in candidatos.columns:
        cluster_counts = candidatos["cluster"].value_counts()

        clusters_ordenados = list(cluster_counts.index)

        # escolhe aleatoriamente um dos top 2 clusters
        TOP_N = min(2, len(clusters_ordenados))
        possiveis_clusters = clusters_ordenados[:TOP_N]

        cluster_escolhido = np.random.choice(possiveis_clusters)

        logs.append(f"[DEBUG] clusters candidatos: {possiveis_clusters} -> escolhido: {cluster_escolhido}")

        candidatos = candidatos[candidatos["cluster"] == cluster_escolhido]

    # fallback para evitar clusters inválidos
    if len(candidatos) == 0:
        logs.append("[WARN] Cluster escolhido ficou vazio, usando todos candidatos sem filtrar cluster.")
        candidatos = aplica_filtros(False, False, False)


    # se ainda assim ficou vazio, volta pros candidatos originais sem filtrar cluster
    if len(candidatos) == 0:
        logs.append("[WARN] Cluster temático ficou vazio, vou ignorar cluster e usar todos os candidatos.")
        candidatos = aplica_filtros(False, False, False)

    if len(candidatos) == 0:
        logs.append("[ERRO] Não há candidatos nem ignorando cluster. Abortando.")
        return {
            "resumo_party": info_party,
            "parametros_pedidos": {
                "dificuldade": dificuldade,
                "tipo": tipo,
                "ambiente": ambiente,
                "tamanho": tamanho,
                "quantidade_oponentes": quantidade_oponentes
            },
            "encontro": pd.DataFrame(),
            "avaliacao": None,
            "logs": logs
        }

    #seleciona os monstros baseados no CR alvo
    escolhidos = escolher_por_cr_alvo(
        df_filtrado=candidatos,
        cr_alvo=cr_alvo_por_inimigo,
        quantidade_oponentes=quantidade_oponentes
    )

    #montar tabela
    cols_show = ["Name", "Type", "Environment", "Size", "CR", "CR_float", "cluster"]
    cols_show = [c for c in cols_show if c in escolhidos.columns]
    encontro_df = escolhidos[cols_show].reset_index(drop=True)

    # avaliar encontro
    avaliacao = avaliar_encontro(
        monstros_df=encontro_df,
        valor_grupo=valor_grupo
    )

    return {
        "resumo_party": info_party,
        "parametros_pedidos": {
            "dificuldade": dificuldade,
            "tipo": tipo,
            "ambiente": ambiente,
            "tamanho": tamanho,
            "quantidade_oponentes": quantidade_oponentes
        },
        "encontro": encontro_df,
        "avaliacao": avaliacao,
        "logs": logs
    }


<h3>Testes</h3>

In [199]:
res = gerar_encontro(
    df_final_refined,
    niveis_jogadores=[3,3,3,3],      #tamanho e niveis da party
    dificuldade="medio",             #pode ser facil / medio / dificil / impossivel
    tipo="any",                      #Tipo, ambiente e tamanho podem ser any
    ambiente="forest",               
    tamanho="medium",                
    quantidade_oponentes=4
)

print("LOGS:")
for l in res["logs"]:
    print(l)

print("\nPARTY / VALOR DO GRUPO:")
print(res["resumo_party"])

print("\nPARAMETROS QUE O USUÁRIO PEDIU:")
print(res["parametros_pedidos"])

print("\nINIMIGOS SUGERIDOS:")
print(res["encontro"])

print("\nAVALIAÇÃO DO ENCONTRO PRA ESSA PARTY:")
print(res["avaliacao"])


LOGS:
[DEBUG] CR base 0.75, multiplicador 'medio'=0.6 => alvo 0.45
[DEBUG] clusters candidatos: [1, 3] -> escolhido: 3

PARTY / VALOR DO GRUPO:
{'qtd_jogadores': 4, 'soma_niveis': 12, 'nivel_medio': 3.0, 'valor_grupo': 3.0}

PARAMETROS QUE O USUÁRIO PEDIU:
{'dificuldade': 'medio', 'tipo': 'any', 'ambiente': 'forest', 'tamanho': 'medium', 'quantidade_oponentes': 4}

INIMIGOS SUGERIDOS:
                               Name   Type                  Environment  \
0        Misc. Creature, Black Bear  Beast               Forest, Plains   
1                  Tri-Flower Frond  Plant                       Forest   
2      Misc. Creature, Giant Badger  Beast  Forest, Plains, Underground   
3  Misc. Creature, Swarm of Insects  Beast               Forest, Plains   

     Size    CR  CR_float  cluster  
0  Medium  0.50      0.50        3  
1  Medium  0.50      0.50        3  
2  Medium  0.25      0.25        3  
3  Medium  0.50      0.50        3  

AVALIAÇÃO DO ENCONTRO PRA ESSA PARTY:
{'soma_nd_mo

In [ ]:
#Variáveis e estados auxiliares
PARTY_LEVEL_LIST = [1]

#Boxes e Layouts
row_layout = Layout(display='flex', flex_flow='row', align_items='stretch', width='100%')
column_layout = Layout(display='flex', flex_flow='column', align_items='stretch', width='100%')
grid_layout = Layout(display='grid', grid_template_columns='repeat(4, 1fr)', grid_gap='10px', width='100%', align_items='stretch')

first_parameter_row = Box(children=[], layout=row_layout)
second_parameter_row = Box(children=[], layout=grid_layout)
party_level_row = Box(children=[], layout=column_layout)
action_button_row = Box(children=[], layout=row_layout)
output_row = Box(children=[], layout=row_layout)

main_layout = Box(children=[first_parameter_row, second_parameter_row, party_level_row, action_button_row], layout=column_layout)

#Utilitários para geração de components 
def make_dropdown(options, value, label, width='100%'): 
  #make dropdown
  drop_layout = Layout(display='flex', flex_flow='column', align_items='stretch', width="95%")
  drop = widgets.Dropdown(options=options, value=value, layout=drop_layout)
  
  #label flutuante
  html_label = widgets.HTML(value=label)
  
  #Layout
  box_layout = Layout(display='flex', flex_flow='column', align_items='stretch', width=width)
  box = Box(children=[html_label, drop], layout=box_layout)
  
  return box

def make_int_field(label, min, max, step=1, value=1, custom_id=None, callback=None, width='100%'):
  int_text_layout = Layout(display='flex', flex_flow='column', align_items='stretch', width="95%")
  int_text = widgets.BoundedIntText(value=value, min=min, max=max, step=step, disabled=False)
  
  if custom_id:
    int_text.custom_id = custom_id
  if callback:
    int_text.observe(on_level_change, names='value')
  
  #label flutuante
  html_label = widgets.HTML(value=label)

  box_layout = Layout(display='flex', flex_flow='column', align_items='stretch', width=width)
  box = Box(children=[html_label, int_text], layout=box_layout)

  return box

#Criando callbacks e funções auxiliáres dos callbacks

def on_level_change(change):
  widget = change['owner']
  new_value = change['new']
  custom_id = getattr(widget, 'custom_id', None)
  if custom_id:
    PARTY_LEVEL_LIST[custom_id-1] = new_value
    print(PARTY_LEVEL_LIST)

def populate_party_row(grow):
  global PARTY_LEVEL_LIST
  editing_box = (len(PARTY_LEVEL_LIST) // 3) + (1 if len(PARTY_LEVEL_LIST) % 3 > 0 else 0)
  if (editing_box < len(party_level_row.children)):
    party_level_row.children = party_level_row.children[:-1]
    return
  elif (editing_box > len(party_level_row.children)):
    party_level_row.children += (Box(children=[], layout=row_layout), )
    party_level_row.children[editing_box-1].children += (make_int_field(f"Nível P{len(PARTY_LEVEL_LIST)}", 1, 20, 1, 1, len(PARTY_LEVEL_LIST), on_level_change, '95%'),)
  else:
    if (grow):
      party_level_row.children[editing_box-1].children += (make_int_field(f"Nível P{len(PARTY_LEVEL_LIST)}", 1, 20, 1, 1, len(PARTY_LEVEL_LIST), on_level_change, '95%'),)
    else:
      party_level_row.children[editing_box-1].children = party_level_row.children[editing_box-1].children[:-1]
      
  
def on_tam_party_change(change):
  global PARTY_LEVEL_LIST
  party_size = change['new']
  old_party_size = change['old']

  if (party_size > old_party_size):
    PARTY_LEVEL_LIST.append(1)
  else:
    PARTY_LEVEL_LIST = PARTY_LEVEL_LIST[:-1]
    
  populate_party_row(party_size > old_party_size)

#Opções para os drops
df_clean = pd.read_csv("outputs/monsters_clean.csv")

listas_ambientes = df_clean['Environment'].str.split(',').apply(lambda list: [item.strip() for item in list])
ambientes = sorted(set(sum(listas_ambientes, [])))

lista_tipos = df_clean['Type'].str.split(',').apply(lambda list: [item.strip() for item in list])
tipos = sorted(set(sum(lista_tipos, [])))

lista_tamanhos = df_clean['Size'].str.split(',').apply(lambda list: [item.strip() for item in list])
tamanhos = sorted(set(sum(lista_tamanhos, [])))

ambientes_options = [(item, item.lower()) for item in ambientes] + [('Todos', 'any')]
dificuldades_options = [('Fácil', 'facil'), ('Médio', 'medio'), ('Difícil', 'dificil'), ('Impossível', 'impossivel')]
tipos_options = [(item, item.lower()) for item in tipos] + [('Todos', 'any')]
tamanhos_options =  [(item, item.lower()) for item in tamanhos] + [('Todos', 'any')]

#Dropdown de dificuldade
drop_dificuldade = make_dropdown(
  options=dificuldades_options,
  value='medio',
  label='Dificuldade',
)
drop_ambiente = make_dropdown(
  options=ambientes_options,
  value='any',
  label='Ambiente',
)
drop_tipo = make_dropdown(
  options=tipos_options,
  value='any',
  label='Tipo dos inimigos',
)
drop_tamanho = make_dropdown(
  options=tamanhos_options,
  value='any',
  label='Tamanho dos inimigos',
)

int_text_tam_party = make_int_field("Número de integrantes", 0, 20, 1, 1)
int_text_num_monstros = make_int_field("Número de oponentes", 0, 20, 1, 1)

button_gerar_combate = widgets.Button(
  description='Gerar combate',
  button_style='info',
  tooltip='Clique para gerar um combate baseado nos dados informados',
)

#Adicionando components às caixas 
first_parameter_row.children = [drop_dificuldade, drop_ambiente, drop_tipo, drop_tamanho]
second_parameter_row.children = [int_text_num_monstros, int_text_tam_party]
populate_party_row(True)
action_button_row.children = [button_gerar_combate]

def handle_gerar_combate_click(b):
  niveis_jogadores = PARTY_LEVEL_LIST
  dificuldade = drop_dificuldade.children[1].value
  tipo = drop_tipo.children[1].value
  ambiente = drop_ambiente.children[1].value
  tamanho = drop_tamanho.children[1].value
  quantidade_oponentes = int_text_num_monstros.children[1].value
  
  res = gerar_encontro(df_final_refined, niveis_jogadores, dificuldade, tipo, ambiente, tamanho, quantidade_oponentes)
  print(res)
  
  linhas = ''
  encontros = res['encontro'].to_numpy().tolist()
  print(res['parametros_pedidos'])
  for i in range(0, len(encontros)):
    celulas = ''
    for j in range(0, 5):
      celulas += f'<td>{str(encontros[i][j])}</td>'
    linhas += f'<tr>{celulas}</tr>'
    
  html_label = widgets.HTML(value=
    f"""
    <style>
        table {{ border-collapse: collapse; width: 96% ; overflow: hidden; }}
        th, td {{ padding: 8px 10px; text-align: left; }}
        tr {{ border-bottom: 1px solid black; solid black}}
        tbody tr:last-child td {{ border-bottom: none}}
        #players-info {{margin-bottom: 24px}}
        #players-info div {{height: 64px; gap: 8px}}
        #players-info div p {{margin: 0}}
        #players-info div h5 {{margin: 0}}
    </style>
    
    <div style="width: 100%; margin-bottom: 24px">
      <h3>Confrontos planejados</h3>
    </div>
    <div id="players-info" style="display: flex; gap: 12px; width: 95%; align-items: center; justify-content: center">
      <div style="width: 20%; border-radius: 8px; border: 1px solid black;display: flex; flex-direction: column; align-items: center; justify-content: center">
        <h5>Número de jogadores</h5>
        <p>{res['resumo_party']['qtd_jogadores']}</p>
      </div>
      <div style="width: 20%; border-radius: 8px; border: 1px solid black;display: flex; flex-direction: column; align-items: center; justify-content: center">
        <h5>Soma dos níveis</h5>
        <p>{res['resumo_party']['soma_niveis']}</p>
      </div>
      <div style="width: 20%; border-radius: 8px; border: 1px solid black;display: flex; flex-direction: column; align-items: center; justify-content: center">
        <h5>Nível médio</h5>
        <p>{res['resumo_party']['nivel_medio']}</p>
      </div>
      <div style="width: 20%; border-radius: 8px; border: 1px solid black;display: flex; flex-direction: column; align-items: center; justify-content: center">
        <h5>Valor do grupo</h5>
        <p>{res['resumo_party']['valor_grupo']}</p>
      </div>
    </div>
    
    <div style="width: 95%; margin-bottom: 24px">
      <h3>Dados esperados do confronto</h3>
      <div style="width: 100%; display: flex; justify-content: space-between">
        <span>Dificuldade: {res['parametros_pedidos']['dificuldade'].capitalize()}</span>
        <span>Tipo: {res['parametros_pedidos']['tipo'].capitalize()}</span>
        <span>Ambiente: {res['parametros_pedidos']['ambiente'].capitalize()}</span>
        <span>Número de oponentes: {res['parametros_pedidos']['quantidade_oponentes']}</span>
      </div>
    </div>
    
    <div style="width: 95%; margin-bottom: 24px">
      <h3>Confrontos sugeridos</h3>
    </div>
    
    <div style="margin-bottom: 24px; background-color:#94a3b8; border-radius: 8px; width: 95%; padding: 2%">
      <table>
        <thead>
          <tr>
            <th>Nome</th>
            <th>Tipo</th>
            <th>Ambiente</th>
            <th>Tamanho</th>
            <th>CR</th>
          </tr>
        </thead>
        <tbody>
          {linhas}
        </tbody>
      </table>
    </div>
    
    <div style="width: 95%; margin-bottom: 24px">
      <h3>Avaliação dos confrontos</h3>
      <div style="display: flex; flex-direction:column; gap: 2px">
        <p style="margin: 0"><b>Nível de Desafio somado:</b> {res['avaliacao']['soma_nd_monstros']}</p>
        <p style="margin: 0"><b>Valor do grupo:</b> {res['avaliacao']['valor_grupo']}</p>
        <p style="margin: 0"><b>Avaliação do nível do encontro:</b> {res['avaliacao']['avaliacao_encontro'].capitalize()}</p>
      </div>
    </div>
    """, layout=row_layout
  )
  
  output_row.children = [html_label]
  if (len(main_layout.children) > 4):
    main_layout.children[4] = output_row
  else:
    main_layout.children += (output_row,)

  print("LOGS:")
  for l in res["logs"]:
      print(l)

  print("\nPARTY / VALOR DO GRUPO:")
  print(res["resumo_party"])

  print("\nPARAMETROS QUE O USUÁRIO PEDIU:")
  print(res["parametros_pedidos"])

  print("\nINIMIGOS SUGERIDOS:")
  print(res["encontro"])

  print("\nAVALIAÇÃO DO ENCONTRO PRA ESSA PARTY:")
  print(res["avaliacao"])

#Aplicando Callbacks
int_text_tam_party.children[1].observe(on_tam_party_change, names='value')

button_gerar_combate.on_click(handle_gerar_combate_click)

main_layout


Box(children=(Box(children=(Box(children=(HTML(value='Dificuldade'), Dropdown(index=1, layout=Layout(align_ite…